# Taller 5 TFIDF Solucion


# Punto 1: Pre-Procesamiento

Leer el archivo `Princesas.csv` usando `pandas` y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [520]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [521]:
data = pd.read_csv('C:/Python/NLP/nlp_projects/Clase_cinco/Princesas.csv', encoding = 'UTF-8')
data 

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [522]:
data['procesado']= data['Personalidad'].apply(lambda fila: fila.lower())
data['procesado']= data['procesado'].apply(lambda fila: re.sub(r"[\W\d_]+"," ",fila))
data['procesado']= data['procesado'].apply(lambda fila: fila.strip())
data['procesado']= data['procesado'].apply(lambda fila: fila.split())
data['procesado']= data['procesado'].apply(lambda fila: [palabra for palabra in fila if palabra not in stopwords_sp])
data['procesado']= data['procesado'].apply(lambda fila: [spanishStemmer.stem(palabra) for palabra in fila])
data['procesado']= data['procesado'].apply(lambda fila:" ".join(fila))
data


,Princesa,Personalidad,procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


# Punto 2: TF-IDF

Crear la matriz TF-IDF

In [523]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(data.procesado.values)

tfidf_matrix = pd.DataFrame(data=tfidf.toarray(), columns=tfidf_vect.get_feature_names())

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix.columns = data['Princesa']

tfidf_matrix

Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.102
acuerd,0.000,0.0,0.056,0.000,0.000,0.000,0.0,0.000,0.08,0.000
ademas,0.000,0.0,0.000,0.000,0.000,0.099,0.0,0.000,0.00,0.000
afortun,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.074,0.000,0.000,0.0,0.000,0.00,0.087
volunt,0.000,0.0,0.000,0.000,0.161,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.122,0.00,0.000


# Punto 3: Distancia del coseno

- Calcular la distancia del coseno entre cada una de las princesas
- ¿Cuáles son las princesas más parecidas?
- ¿Cuáles son las princesas más diferentes?

In [524]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos



Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Princesa,,,,,,,,,,
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818


In [525]:
dist_cos.max()

Princesa
Blancanieves    0.985136
Cenicienta      0.974925
Aurora          0.965930
Bella           0.973338
Jasmín          0.972505
Pocahontas      0.979646
Mulan           0.991170
Tiana           0.991170
Mérida          0.987267
Moana           0.974925
dtype: float64

In [527]:
for col in dist_cos:
    print(col)
    print(max(dist_cos[col]))
    print(" ")

Blancanieves
0.9851355494133917
 
Cenicienta
0.9749254322778309
 
Aurora
0.9659301373531197
 
Bella
0.9733380234495987
 
Jasmín
0.9725047830960202
 
Pocahontas
0.9796455714861364
 
Mulan
0.9911697927686938
 
Tiana
0.9911697927686938
 
Mérida
0.9872667627061668
 
Moana
0.9749254322778309
 


**Las princesas menos parecidas son:**

- Blancanieves y Merida
- Cenienta y Moana
- Aurora y Moana
- Bella y Tiana
- Jasmin y Merida
- Pocahontas y Merida
- Mulan y Tiana
- Tiana y Mulan
- Merida y Mulan
- Moana y Cenicienta

**Las princesas mas parecidas son:**

- Blancanieves y Aurora
- Cenicienta y Blancanieves
- Aurora y Bella
- Bella y Aurora
- Jasmín y Bella
- Pocahontas y Aurora
- Mulan y Bella
- Tiana y Cenicienta
- Mérida y Aurora
- Moana y Bella
